In [ ]:
import json
from tqdm import tqdm
import gcsfs
import tempfile
from google.cloud import storage
import pandas as pd
import os
from ultralytics import YOLO

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/content/drive/MyDrive/code/MJ/dlproject-463310-0903d7003ee1.json'

In [ ]:
# 1. YOLOv8 설치 (Ultralytics GitHub의 최신 버전 개발 브랜치 경로 사용)
!pip install git+https://github.com/ultralytics/ultralytics.git@main
!pip install gcsfs
!pip install scikit-learn --quiet

  Cloning https://github.com/ultralytics/ultralytics.git (to revision main) to /tmp/pip-req-build-2y49wsqb
  Running command git clone --filter=blob:none --quiet https://github.com/ultralytics/ultralytics.git /tmp/pip-req-build-2y49wsqb
  Resolved https://github.com/ultralytics/ultralytics.git to commit 1f7c05ec0a2a3d63964af1245b53289323037e0c
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 126.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 101.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 63.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# 2. GCS 파일 탐색 및 확인
bucket_name = 'deeplearn-bucket-1'
client = storage.Client()

# GCS 연결
fs = gcsfs.GCSFileSystem()

# 버킷 이름
bucket_name = 'deeplearn-bucket-1'

# GCS 내부 경로 목록 (이미지 경로)
prefixes = [
    "Training/01.원천데이터/TS_task1_yolo_dispose_BLOUSE_unzipped/TS_task1_yolo_dispose_BLOUSE_unzipped",
    "Training/01.원천데이터/TS_task1_yolo_dispose_SHIRTS_unzipped/TS_task1_yolo_dispose_SHIRTS_unzipped",
    "Training/01.원천데이터/TS_task1_yolo_recycle_BLOUSE_unzipped/TS_task1_yolo_recycle_BLOUSE_unzipped",
    "Training/01.원천데이터/TS_task1_yolo_recycle_SHIRTS_unzipped/TS_task1_yolo_recycle_SHIRTS_unzipped",
    "Training/01.원천데이터/TS_task1_yolo_reusable_BLOUSE_unzipped/TS_task1_yolo_reusable_BLOUSE_unzipped",
    "Training/01.원천데이터/TS_task1_yolo_reusable_SHIRTS_unzipped/TS_task1_yolo_reusable_SHIRTS_unzipped"
]

# 결과 저장 리스트
results = []

# 각 prefix에 대해 반복 수행
for prefix in prefixes:
    full_path = f"{bucket_name}/{prefix}"
    files = fs.ls(full_path)

    # jpg/png 개수 세기
    jpg_count = sum(1 for f in files if f.endswith('.jpg'))
    png_count = sum(1 for f in files if f.endswith('.png'))
    total = jpg_count + png_count

    results.append({
        "prefix": prefix,
        "jpg": jpg_count,
        "png": png_count,
        "total": total
    })

# 결과 출력
for i, res in enumerate(results, 1):
    print(f"[{i}] {res['prefix']}")
    print(f"총 이미지 개수: {res['total']} (jpg: {res['jpg']}, png: {res['png']})\n")

[1] Training/01.원천데이터/TS_task1_yolo_dispose_BLOUSE_unzipped/TS_task1_yolo_dispose_BLOUSE_unzipped
총 이미지 개수: 438 (jpg: 438, png: 0)

[2] Training/01.원천데이터/TS_task1_yolo_dispose_SHIRTS_unzipped/TS_task1_yolo_dispose_SHIRTS_unzipped
총 이미지 개수: 520 (jpg: 520, png: 0)

[3] Training/01.원천데이터/TS_task1_yolo_recycle_BLOUSE_unzipped/TS_task1_yolo_recycle_BLOUSE_unzipped
총 이미지 개수: 1184 (jpg: 1184, png: 0)

[4] Training/01.원천데이터/TS_task1_yolo_recycle_SHIRTS_unzipped/TS_task1_yolo_recycle_SHIRTS_unzipped
총 이미지 개수: 1198 (jpg: 1198, png: 0)

[5] Training/01.원천데이터/TS_task1_yolo_reusable_BLOUSE_unzipped/TS_task1_yolo_reusable_BLOUSE_unzipped
총 이미지 개수: 2074 (jpg: 2074, png: 0)

[6] Training/01.원천데이터/TS_task1_yolo_reusable_SHIRTS_unzipped/TS_task1_yolo_reusable_SHIRTS_unzipped
총 이미지 개수: 4142 (jpg: 4142, png: 0)



In [ ]:
# 3-1. YOLOv8 학습을 위해 라벨 데이터 .json -> .txt 변환 함수 정의
fs = gcsfs.GCSFileSystem()

def convert_label(json_blob_path, save_txt_path, class_id=0):
    with fs.open(json_blob_path, 'r') as f:
        data = json.load(f)

    annotations = data.get("annotations", [])
    if not annotations:
        return

    txt_lines = []

    for ann in annotations:
        bbox = ann.get("bbox", [])
        if len(bbox) == 4:
            x, y, w, h = bbox
            x_center = x + w / 2
            y_center = y + h / 2
            txt_line = f"{class_id} {x_center:.6f} {y_center:.6f} {w:.6f} {h:.6f}"
            txt_lines.append(txt_line)

    with open(save_txt_path, "w") as out:
        out.write("\n".join(txt_lines))

In [ ]:
# 3-2. .json -> .txt 변환
import os
import json
import gcsfs
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor

# GCS 설정
fs = gcsfs.GCSFileSystem()

# 결과 저장 루트 경로
root_dir = "/content/drive/MyDrive/Colab Notebooks/yolo_dataset"
img_dir = os.path.join(root_dir, "images/data")
label_dir = os.path.join(root_dir, "labels/data")

# 카테고리 목록
categories = [
    ("dispose_BLOUSE", 0),
    ("dispose_SHIRTS", 1),
    ("recycle_BLOUSE", 2),
    ("recycle_SHIRTS", 3),
    ("reusable_BLOUSE", 4),
    ("reusable_SHIRTS", 5)
]

# YOLO 변환 함수 정의
def convert_label(json_blob_path, save_txt_path, class_id):
    with fs.open(json_blob_path, 'r') as f:
        content = f.read()
        data = json.loads(content)

    # 이미지 크기 정보
    width = data["meta_information"]["width"]
    height = data["meta_information"]["height"]

    annotations = data.get("annotations", {})
    if not annotations:
        return

    txt_lines = []

    for category in ["clothes", "pollution"]:
        objects = annotations.get(category, [])
        for obj in objects:
            bboxes = obj.get("bbox", [])
            for bbox in bboxes:
                if len(bbox) == 4:
                    x1, y1, x2, y2 = bbox
                    w = x2 - x1
                    h = y2 - y1
                    x_center = x1 + w / 2
                    y_center = y1 + h / 2

                    txt_line = f"{class_id} {x_center/width:.6f} {y_center/height:.6f} {w/width:.6f} {h/height:.6f}"
                    txt_lines.append(txt_line)

    if txt_lines:
        with open(save_txt_path, "w") as out:
            out.write("\n".join(txt_lines))

# 개별 이미지 처리 함수
def process_single_file(args):
    img_path, img_prefix, json_prefix, label_dir, img_dir, class_id = args
    filename = os.path.basename(img_path)
    if not filename.lower().endswith(".jpg"):
        return

    base = filename.replace(".jpg", "")
    json_path = f"{json_prefix}/{base}.json"
    txt_path = os.path.join(label_dir, f"{base}.txt")
    local_img_path = os.path.join(img_dir, filename)

    try:
        fs.get(img_path, local_img_path)
        if fs.exists(json_path):
            convert_label(json_path, txt_path, class_id=class_id)
    except Exception as e:
        print(f"error - {filename}: {e}")

# 카테고리별 처리 함수
def process_category(name, class_id):
    print(f"\n 처리 중: {name} (class_id={class_id})")

    # GCS 경로
    img_prefix = f"deeplearn-bucket-1/Training/01.원천데이터/TS_task1_yolo_{name}_unzipped/TS_task1_yolo_{name}_unzipped"
    json_prefix = f"deeplearn-bucket-1/Training/02.라벨링데이터/TL_task1_yolo_{name}_unzipped/TL_task1_yolo_{name}_unzipped"

    # 로컬 폴더 생성
    os.makedirs(img_dir, exist_ok=True)
    os.makedirs(label_dir, exist_ok=True)

    # 이미지 리스트 가져오기
    try:
        images = fs.ls(img_prefix)
    except Exception as e:
        print(f"이미지 목록 가져오기 실패: {img_prefix}")
        return

    print(f"[{name}] 총 이미지 수: {len(images)}")

    # 병렬 처리
    args_list = [
        (img_path, img_prefix, json_prefix, label_dir, img_dir, class_id)
        for img_path in images
    ]

    with ThreadPoolExecutor(max_workers=8) as executor:
        list(tqdm(executor.map(process_single_file, args_list), total=len(args_list)))

    print(f"완료: {name}\n")

# 전체 실행
for name, class_id in categories:
    process_category(name, class_id)


 처리 중: dispose_BLOUSE (class_id=0)
[dispose_BLOUSE] 총 이미지 수: 438


100%|██████████| 438/438 [02:19<00:00,  3.14it/s]


완료: dispose_BLOUSE


 처리 중: dispose_SHIRTS (class_id=1)
[dispose_SHIRTS] 총 이미지 수: 520


100%|██████████| 520/520 [01:51<00:00,  4.66it/s]


완료: dispose_SHIRTS


 처리 중: recycle_BLOUSE (class_id=2)
[recycle_BLOUSE] 총 이미지 수: 1184


...

완료: recycle_SHIRTS


 처리 중: reusable_BLOUSE (class_id=4)
[reusable_BLOUSE] 총 이미지 수: 2074


100%|██████████| 2074/2074 [04:46<00:00,  7.23it/s]


완료: reusable_BLOUSE


 처리 중: reusable_SHIRTS (class_id=5)
[reusable_SHIRTS] 총 이미지 수: 4142


100%|██████████| 4142/4142 [09:11<00:00,  7.50it/s]

완료: reusable_SHIRTS



In [ ]:
import random
import os
import shutil
from concurrent.futures import ThreadPoolExecutor
from sklearn.model_selection import train_test_split

# 4. train/val 분할
# 경로 설정
root_dir = "/content/drive/MyDrive/Colab Notebooks/yolo_dataset"
img_data_dir = os.path.join(root_dir, "images/data")
label_data_dir = os.path.join(root_dir, "labels/data")

# 이미지 목록
all_images = os.listdir(img_data_dir)
image_paths = [f for f in all_images if f.endswith(".jpg")]

# train / val 8:2 split
train_imgs, val_imgs = train_test_split(image_paths, test_size=0.2, random_state=42)

# 디렉토리 생성
for phase in ["train", "val"]:
    os.makedirs(os.path.join(root_dir, "images", phase), exist_ok=True)
    os.makedirs(os.path.join(root_dir, "labels", phase), exist_ok=True)

# 병렬 복사 함수
def copy_pair(img_name, phase):
    # 이미지 경로
    src_img = os.path.join(img_data_dir, img_name)
    dst_img = os.path.join(root_dir, "images", phase, img_name)
    shutil.copyfile(src_img, dst_img)

    # 라벨 경로
    label_name = img_name.replace(".jpg", ".txt")
    src_label = os.path.join(label_data_dir, label_name)
    dst_label = os.path.join(root_dir, "labels", phase, label_name)
    if os.path.exists(src_label):
        shutil.copyfile(src_label, dst_label)

# 병렬 실행
for img_list, phase in zip([train_imgs, val_imgs], ["train", "val"]):
    with ThreadPoolExecutor(max_workers=8) as executor:
        executor.map(lambda img: copy_pair(img, phase), img_list)

In [ ]:
yaml_path = "/content/temp_yolo_dataset/data.yaml"

with open(yaml_path, "w") as f:
    f.write(f"""
path: /content/temp_yolo_dataset
train: images/train
val: images/val

names:
  0: dispose_BLOUSE
  1: dispose_SHIRTS
  2: recycle_BLOUSE
  3: recycle_SHIRTS
  4: reusable_BLOUSE
  5: reusable_SHIRTS
""")

In [ ]:
yaml_path = "/content/temp_yolo_dataset/data.yaml"

with open(yaml_path, "r") as f:
    content = f.read()

print(content)


path: /content/temp_yolo_dataset
train: images/train
val: images/val

names:
  0: dispose_BLOUSE
  1: dispose_SHIRTS
  2: recycle_BLOUSE
  3: recycle_SHIRTS
  4: reusable_BLOUSE
  5: reusable_SHIRTS



In [ ]:
# 링크 생성
!ln -s "/content/drive/MyDrive/code/MJ/yolo_dataset/images/train" /content/temp_yolo_dataset/images/train
!ln -s "/content/drive/MyDrive/code/MJ/yolo_dataset/images/val" /content/temp_yolo_dataset/images/val
!ln -s "/content/drive/MyDrive/code/MJ/yolo_dataset/labels/train" /content/temp_yolo_dataset/labels/train
!ln -s "/content/drive/MyDrive/code/MJ/yolo_dataset/labels/val" /content/temp_yolo_dataset/labels/val

In [ ]:
!mkdir -p /content/drive/MyDrive/code/MJ/yolo_dataset/labels/val

In [ ]:
from ultralytics import YOLO
import os

yaml_path = "/content/temp_yolo_dataset/data.yaml"

with open(yaml_path, "w") as f:
    f.write(f"""
path: /content/temp_yolo_dataset
train: images/train
val: images/val

names:
  0: dispose_BLOUSE
  1: dispose_SHIRTS
  2: recycle_BLOUSE
  3: recycle_SHIRTS
  4: reusable_BLOUSE
  5: reusable_SHIRTS
""")

model = YOLO("yolov8n.pt")  # 경량 모델

model.train(
    data=yaml_path,
    epochs=40,
    imgsz=416,
    batch=64,
    workers=4,
    device=0,
    project="/content/drive/MyDrive/code/MJ/yolo_dataset/runs",
    name="yolov8_train",
    exist_ok=True,
    save_period=1,
    amp=True,
    pretrained=True,
    patience=15,
    cache=True
)

Ultralytics 8.3.156 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=64, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/temp_yolo_dataset/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=40, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=416, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov8_train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=15, perspective=0.0, plots=True,

train: Scanning /content/drive/.shortcut-targets-by-id/1JPrsNv3kKRcuN6rPQKQ0UXU8RRcxpfm3/yolo_dataset/labels/train.cache... 7628 images, 0 backgrounds, 0 corrupt: 100%|██████████| 7628/7628 [00:00<?, ?it/s]


WARNING ⚠️ cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.


train: Caching images (2.8GB RAM): 100%|██████████| 7628/7628 [02:16<00:00, 55.78it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access ✅ (ping: 0.6±0.0 ms, read: 403.2±116.6 MB/s, size: 1940.0 KB)


...

                   all       1907       2323      0.719      0.723      0.732      0.708



40 epochs completed in 0.249 hours.
Optimizer stripped from /content/drive/MyDrive/code/MJ/yolo_dataset/runs/yolov8_train/weights/last.pt, 6.2MB
Optimizer stripped from /content/drive/MyDrive/code/MJ/yolo_dataset/runs/yolov8_train/weights/best.pt, 6.2MB

Validating /content/drive/MyDrive/code/MJ/yolo_dataset/runs/yolov8_train/weights/best.pt...
Ultralytics 8.3.156 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
Model summary (fused): 72 layers, 3,006,818 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 15/15 [00:07<00:00,  1.91it/s]


                   all       1907       2323       0.75      0.713       0.74      0.717
        dispose_BLOUSE         90        278      0.571      0.311       0.37        0.3
        dispose_SHIRTS        104        332      0.635      0.328      0.369        0.3
        recycle_BLOUSE        241        241      0.754        0.9      0.882       0.88
        recycle_SHIRTS        240        240      0.802      0.858      0.892      0.892
       reusable_BLOUSE        416        416      0.823      0.923      0.954      0.954
       reusable_SHIRTS        816        816      0.912       0.96      0.975      0.975
Speed: 0.0ms preprocess, 0.2ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to /content/drive/MyDrive/code/MJ/yolo_dataset/runs/yolov8_train


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7fb99b68df50>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
     

In [ ]:
# 학습 결과 평가
metrics = model.val()  # validation set에서 평가
print(metrics.box.map)  # box mAP 출력

Ultralytics 8.3.156 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
Model summary (fused): 72 layers, 3,006,818 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.5±0.2 ms, read: 606.3±182.8 MB/s, size: 2365.3 KB)


val: Scanning /content/drive/.shortcut-targets-by-id/1JPrsNv3kKRcuN6rPQKQ0UXU8RRcxpfm3/yolo_dataset/labels/val.cache... 1907 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1907/1907 [00:00<?, ?it/s]


WARNING ⚠️ cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.


val: Caching images (0.7GB RAM): 100%|██████████| 1907/1907 [00:33<00:00, 56.25it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 30/30 [00:08<00:00,  3.73it/s]


                   all       1907       2323      0.749      0.713       0.74      0.717
        dispose_BLOUSE         90        278      0.571      0.311      0.369        0.3
        dispose_SHIRTS        104        332      0.632      0.328       0.37      0.301
        recycle_BLOUSE        241        241      0.754        0.9      0.882       0.88
        recycle_SHIRTS        240        240      0.798      0.854      0.892      0.892
       reusable_BLOUSE        416        416      0.823      0.923      0.954      0.954
       reusable_SHIRTS        816        816      0.913       0.96      0.975      0.975
Speed: 0.2ms preprocess, 0.4ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to /content/drive/MyDrive/code/MJ/yolo_dataset/runs/yolov8_train
0.7168934531264072


In [ ]:
from IPython.display import Image, display
import os
import random

# 학습된 결과가 저장된 디렉토리
result_dir = '/content/drive/MyDrive/code/MJ/yolo_dataset/runs/yolov8_train/predict'

# 예측된 이미지 중 임의로 3장 출력
predicted_images = [img for img in os.listdir(result_dir) if img.endswith('.jpg') or img.endswith('.png')]
sample_images = random.sample(predicted_images, 3)

# 이미지 표시
for img_file in sample_images:
    display(Image(filename=os.path.join(result_dir, img_file)))

In [ ]:
# 예측 테스트 예시
results = model.predict(("/content/drive/MyDrive/code/MJ/yolo_dataset/images/val"), save=True, conf=0.25)
results.head()


WARNING ⚠️ 
inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

image 1/1907 /content/drive/MyDrive/code/MJ/yolo_dataset/images/val/DP_BL_00086.jpg: 416x320 1 recycle_BLOUSE, 49.1ms
image 2/1907 /content/drive/MyDrive/code/MJ/yolo_dataset/images/val/DP_BL_00091.jpg: 416x320 1 recycle_BLOUSE, 8.2ms
image 3/1907 /content/drive/MyDrive/code/MJ/yolo_dataset/images/val/DP_BL_00094.jpg: 416x320 1 dispose_BLOUSE, 8.1ms
image 4/1907 /content/drive/MyDrive/code/MJ/yolo_dataset/images/val/DP_BL_00097.jpg: 416x320 1 dispose_B

KeyboardInterrupt: 

In [ ]:
import os
import gcsfs
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor

# GCS 연결
fs = gcsfs.GCSFileSystem()

# 로컬 YOLOv8 결과 폴더 경로
root_dir = "/content/drive/MyDrive/code/MJ/yolo_dataset"
gcs_target_prefix = "deeplearn-bucket-1/YOLOv8"

# 업로드할 서브폴더 목록
subfolders = [
    "runs/yolov8_train/weights"
]

# 업로드 함수
def upload_file(local_path, gcs_path):
    try:
        with open(local_path, "rb") as f_local:
            with fs.open(gcs_path, "wb") as f_gcs:
                f_gcs.write(f_local.read())
        return True
    except Exception as e:
        print(f"\n업로드 실패: {local_path} -> {gcs_path}, 오류: {e}")
        return False

# 업로드할 작업 목록 만들기
upload_tasks = []
for subfolder in subfolders:
    local_path = os.path.join(root_dir, subfolder)

    if os.path.isfile(local_path):
        gcs_path = f"{gcs_target_prefix}/{subfolder}"
        upload_tasks.append((local_path, gcs_path))
        continue

    for file in os.listdir(local_path):
        local_file_path = os.path.join(local_path, file)
        gcs_file_path = f"{gcs_target_prefix}/{subfolder}/{file}"
        upload_tasks.append((local_file_path, gcs_file_path))

# 툴바 + 병렬 업로드
results = []
with ThreadPoolExecutor(max_workers=8) as executor:
    with tqdm(total=len(upload_tasks), desc="GCS 업로드 중", ncols=100) as pbar:
        futures = [executor.submit(upload_file, *task) for task in upload_tasks]
        for future in futures:
            result = future.result()
            results.append(result)
            pbar.update(1)

# 요약 출력
uploaded_count = sum(results)
total_files = len(results)
print(f"\n업로드 완료: {uploaded_count}/{total_files} 파일")

GCS 업로드 중: 100%|████████████████████████████████████████████████| 42/42 [00:24<00:00,  1.69it/s]


업로드 완료: 42/42 파일


# 성능 개선 시도 1

In [ ]:
model = YOLO("yolov8n.pt")  # 경량 모델

model.train(
    data=yaml_path,
    epochs=10, # 학습 반복 횟수
    imgsz=320, # 입력 이미지 크기 (320x320)
    batch=64, # 한 번에 처리할 이미지 수
    workers=4,
    device=0,
    project=os.path.join(root_dir, "runs"),
    name="yolov8_train",
    exist_ok=True,
    save_period=1,
    amp=True,
    pretrained=True,
    patience=10, # early stopping 조건 (10 epoch 동안 성능 개선 없으면 중단)
    cache=True
)

Ultralytics 8.3.156 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=64, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/temp_yolo_dataset/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=320, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov8_train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=10, perspective=0.0, plots=True,

train: Scanning /content/drive/.shortcut-targets-by-id/1JPrsNv3kKRcuN6rPQKQ0UXU8RRcxpfm3/yolo_dataset/labels/train.cache... 7628 images, 0 backgrounds, 0 corrupt: 100%|██████████| 7628/7628 [00:00<?, ?it/s]


WARNING ⚠️ cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.


train: Caching images (1.6GB RAM): 100%|██████████| 7628/7628 [03:31<00:00, 36.14it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


...

                   all       1907       2323      0.697       0.62       0.64      0.627



10 epochs completed in 0.062 hours.
Optimizer stripped from /content/drive/MyDrive/code/MJ/yolo_dataset/runs_1/yolov8_train/weights/last.pt, 6.2MB
Optimizer stripped from /content/drive/MyDrive/code/MJ/yolo_dataset/runs_1/yolov8_train/weights/best.pt, 6.2MB

Validating /content/drive/MyDrive/code/MJ/yolo_dataset/runs_1/yolov8_train/weights/best.pt...
Ultralytics 8.3.156 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
Model summary (fused): 72 layers, 3,006,818 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 15/15 [00:07<00:00,  1.93it/s]


                   all       1907       2323      0.627      0.652      0.653      0.641
        dispose_BLOUSE         90        278      0.501      0.185      0.224      0.185
        dispose_SHIRTS        104        332      0.479      0.181      0.218      0.188
        recycle_BLOUSE        241        241      0.599      0.925      0.765      0.764
        recycle_SHIRTS        240        240      0.705       0.75      0.825      0.825
       reusable_BLOUSE        416        416      0.553      0.974      0.924      0.922
       reusable_SHIRTS        816        816      0.922        0.9      0.961      0.961
Speed: 0.0ms preprocess, 0.1ms inference, 0.0ms loss, 1.3ms postprocess per image
Results saved to /content/drive/MyDrive/code/MJ/yolo_dataset/runs_1/yolov8_train


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7fb998135750>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
     

In [ ]:
# 학습 결과 평가
metrics = model.val()  # validation set에서 평가
print(metrics.box.map)  # box mAP 출력

Ultralytics 8.3.156 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
Model summary (fused): 72 layers, 3,006,818 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.4±0.0 ms, read: 634.3±155.3 MB/s, size: 2365.3 KB)


val: Scanning /content/drive/.shortcut-targets-by-id/1JPrsNv3kKRcuN6rPQKQ0UXU8RRcxpfm3/yolo_dataset/labels/val.cache... 1907 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1907/1907 [00:00<?, ?it/s]


WARNING ⚠️ cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.


val: Caching images (0.4GB RAM): 100%|██████████| 1907/1907 [00:33<00:00, 57.55it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 30/30 [00:07<00:00,  3.99it/s]


                   all       1907       2323      0.627      0.652      0.653      0.641
        dispose_BLOUSE         90        278      0.503      0.186      0.225      0.185
        dispose_SHIRTS        104        332       0.48      0.181      0.218      0.188
        recycle_BLOUSE        241        241      0.599      0.925      0.766      0.764
        recycle_SHIRTS        240        240      0.705       0.75      0.825      0.825
       reusable_BLOUSE        416        416      0.554      0.974      0.924      0.922
       reusable_SHIRTS        816        816      0.923      0.899      0.961      0.961
Speed: 0.1ms preprocess, 0.4ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to /content/drive/MyDrive/code/MJ/yolo_dataset/runs_1/yolov8_train
0.6411347234484629


# 성능 개선 시도 2

In [ ]:
model = YOLO("yolov8n.pt")  # 경량 모델

model.train(
    data=yaml_path,
    epochs=40,
    imgsz=640,
    batch=64,
    workers=4,
    device=0,
    project="/content/drive/MyDrive/code/MJ/yolo_dataset/runs_2",
    name="yolov8_train",
    exist_ok=True,
    save_period=1,
    amp=True,
    pretrained=True,
    patience=10,
    cache=True
)

Ultralytics 8.3.156 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=64, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/temp_yolo_dataset/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=40, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov8_train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=10, perspective=0.0, plots=True,

train: Scanning /content/drive/.shortcut-targets-by-id/1JPrsNv3kKRcuN6rPQKQ0UXU8RRcxpfm3/yolo_dataset/labels/train.cache... 7628 images, 0 backgrounds, 0 corrupt: 100%|██████████| 7628/7628 [00:00<?, ?it/s]


WARNING ⚠️ cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.


train: Caching images (6.5GB RAM): 100%|██████████| 7628/7628 [03:43<00:00, 34.09it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access ✅ (ping: 0.5±0.0 ms, read: 453.8±122.5 MB/s, size: 1940.0 KB)


...

                   all       1907       2323      0.743      0.723      0.751      0.721



40 epochs completed in 0.382 hours.
Optimizer stripped from /content/drive/MyDrive/code/MJ/yolo_dataset/runs_2/yolov8_train/weights/last.pt, 6.2MB
Optimizer stripped from /content/drive/MyDrive/code/MJ/yolo_dataset/runs_2/yolov8_train/weights/best.pt, 6.2MB

Validating /content/drive/MyDrive/code/MJ/yolo_dataset/runs_2/yolov8_train/weights/best.pt...
Ultralytics 8.3.156 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
Model summary (fused): 72 layers, 3,006,818 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 15/15 [00:08<00:00,  1.76it/s]


                   all       1907       2323      0.753      0.717      0.752      0.723
        dispose_BLOUSE         90        278      0.623       0.32        0.4      0.302
        dispose_SHIRTS        104        332      0.569      0.337      0.385      0.315
        recycle_BLOUSE        241        241      0.754      0.888      0.889      0.886
        recycle_SHIRTS        240        240      0.759      0.883      0.904      0.904
       reusable_BLOUSE        416        416      0.906      0.883      0.964      0.964
       reusable_SHIRTS        816        816      0.906       0.99      0.968      0.968
Speed: 0.1ms preprocess, 0.3ms inference, 0.0ms loss, 1.1ms postprocess per image
Results saved to /content/drive/MyDrive/code/MJ/yolo_dataset/runs_2/yolov8_train


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7fb99b87ba50>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
     

In [ ]:
# 학습 결과 평가
metrics = model.val()  # validation set에서 평가
print(metrics.box.map)  # box mAP 출력

Ultralytics 8.3.156 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
Model summary (fused): 72 layers, 3,006,818 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.4±0.0 ms, read: 630.1±180.3 MB/s, size: 2365.3 KB)


val: Scanning /content/drive/.shortcut-targets-by-id/1JPrsNv3kKRcuN6rPQKQ0UXU8RRcxpfm3/yolo_dataset/labels/val.cache... 1907 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1907/1907 [00:00<?, ?it/s]


WARNING ⚠️ cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.


val: Caching images (1.6GB RAM): 100%|██████████| 1907/1907 [00:35<00:00, 53.10it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 30/30 [00:11<00:00,  2.72it/s]


                   all       1907       2323      0.752      0.718      0.752      0.723
        dispose_BLOUSE         90        278      0.623       0.32      0.401      0.301
        dispose_SHIRTS        104        332      0.565      0.337      0.385      0.314
        recycle_BLOUSE        241        241      0.755      0.888      0.889      0.887
        recycle_SHIRTS        240        240      0.755      0.887      0.904      0.904
       reusable_BLOUSE        416        416      0.906      0.883      0.964      0.964
       reusable_SHIRTS        816        816      0.906       0.99      0.967      0.967
Speed: 0.5ms preprocess, 0.5ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to /content/drive/MyDrive/code/MJ/yolo_dataset/runs_2/yolov8_train
0.7229225122688887


In [ ]:
import os
import gcsfs
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor

# GCS 연결
fs = gcsfs.GCSFileSystem()

# 로컬 YOLOv8 결과 폴더 경로
root_dir = "/content/drive/MyDrive/code/MJ/yolo_dataset"
gcs_target_prefix = "deeplearn-bucket-1/YOLOv8"

# 업로드할 서브폴더 목록
subfolders = [
    "runs_1/yolov8_train/weights",
    "runs_2/yolov8_train/weights"
]

# 업로드 함수
def upload_file(local_path, gcs_path):
    try:
        with open(local_path, "rb") as f_local:
            with fs.open(gcs_path, "wb") as f_gcs:
                f_gcs.write(f_local.read())
        return True
    except Exception as e:
        print(f"\n업로드 실패: {local_path} -> {gcs_path}, 오류: {e}")
        return False

# 업로드할 작업 목록 만들기
upload_tasks = []
for subfolder in subfolders:
    local_path = os.path.join(root_dir, subfolder)

    if os.path.isfile(local_path):
        gcs_path = f"{gcs_target_prefix}/{subfolder}"
        upload_tasks.append((local_path, gcs_path))
        continue

    for file in os.listdir(local_path):
        local_file_path = os.path.join(local_path, file)
        gcs_file_path = f"{gcs_target_prefix}/{subfolder}/{file}"
        upload_tasks.append((local_file_path, gcs_file_path))

# 툴바 + 병렬 업로드
results = []
with ThreadPoolExecutor(max_workers=8) as executor:
    with tqdm(total=len(upload_tasks), desc="GCS 업로드 중", ncols=100) as pbar:
        futures = [executor.submit(upload_file, *task) for task in upload_tasks]
        for future in futures:
            result = future.result()
            results.append(result)
            pbar.update(1)

# 요약 출력
uploaded_count = sum(results)
total_files = len(results)
print(f"\n업로드 완료: {uploaded_count}/{total_files} 파일")

GCS 업로드 중: 100%|████████████████████████████████████████████████| 54/54 [00:31<00:00,  1.73it/s]


업로드 완료: 54/54 파일
